## **2.1 En C++**

Este ejemplo realiza la multiplicación de una matriz por un vector. La cantidad de elementos del vector y la matriz esta dada por la cantidad de procesos con que se ejecuta el programa. Por ejemplo si se ejecuta con 4 procesos, entonces el vector va a ser 4 elementos y martriz va a ser de 4*4

In [ ]:
code= """
#include <mpi.h>
#include <iostream>

using namespace std;

void rellenar_vector(int vec[], int cant)
{
  for(int i=0;i<cant;i++)
  {
    vec[i]=rand()%5;
  }
}

void mostrar_vector(int vec[], int cant)
{
  cout<<"Mostrando vector"<<endl;
  cout<<"(";
  for(int i=0;i<cant;i++)
  {
    cout<<vec[i]<<" ";
  }
  cout<<")"<<endl;
}


void rellenar_matriz(int matriz[], int cant_filas,int cant_col)
{
  for(int i=0; i<cant_filas;i++)
  {
    for(int j=0; j<cant_col;j++)
    {
      matriz[i*cant_col+j]=rand()%5;
    }
  }
}


void mostrar_matriz(int matriz[], int cant_filas,int cant_col)
{
  cout<<"Mostrando Matriz"<<endl;

  for(int i=0; i<cant_filas;i++)
  {
    for(int j=0; j<cant_col;j++)
    {
      cout<<matriz[i*cant_col+j]<<" ";
    }
    cout<<endl;
  }
}
void crear_vector(int **vec, int cant)
{
  *vec = new int[cant];
}

int main(int argc, char** argv) 
{
        int rank;
        int buf;
        int nproc;
        const int root=0;


        srand(time(NULL));
                    
        MPI_Init(&argc, &argv);
        MPI_Comm_size(MPI_COMM_WORLD, &nproc );
        MPI_Comm_rank(MPI_COMM_WORLD, &rank);

        
        
        int *vector,*matriz,*resultado;
        int *mat_aux_col;// cuando se hace el scatter aca se copian los datos de cada fila, por cada proceso.
        int cant_filas=nproc;
        int cant_col=nproc;

        //Como vector y mat_aux_col lo van a tener que usar en scatter y en el bcast todos los procesos, 
        //en donde se van hacer las copias. Entonces se genera el new fuera del if
        crear_vector(&vector,nproc);        
        crear_vector(&mat_aux_col,cant_col);
        
        if(rank==0)
        {
          //Como esta parte solamente el master, se declara en esta parte
          rellenar_vector(vector,nproc);
          mostrar_vector(vector,nproc);
          
          //como matriz solamente se va a usar como dato de lectura se crea el new en el master
          crear_vector(&matriz,cant_filas*cant_col);
          rellenar_matriz(matriz,cant_filas,cant_col);
          mostrar_matriz(matriz,cant_filas,cant_col);

          //como resultado solamente lo va escribrir el padre entonces se el new dentro del if
          crear_vector(&resultado,nproc);
        }
       
        //distribuyo cada fila de la matriz en un proceso distinto
        MPI_Scatter(matriz,cant_col,MPI_INT,mat_aux_col,cant_col,MPI_INT,0,MPI_COMM_WORLD);
        //mostrar_vector(mat_aux_col,cant_col); 

        //genero copias del vector en cada proceso
        MPI_Bcast(vector,nproc,MPI_INT,0,MPI_COMM_WORLD);
        //mostrar_vector(vector,nproc);
        
        MPI_Barrier(MPI_COMM_WORLD);

        int subfinal=0;

        //realizo la multiplicacion de cada fila por el vector
        for(int i=0; i<nproc;i++)
        {
          subfinal+=mat_aux_col[i]*vector[i];
        }

        //cout<<subfinal<<endl;

       //el resultado que obtuvo cada proceso al realizar la multiplicacion lo junto en el vector resultado
       MPI_Gather(&subfinal,1,MPI_INT,resultado,1,MPI_INT,0,MPI_COMM_WORLD);

        if(rank==0)
        {
           mostrar_vector(resultado,nproc);
           delete [] matriz;
           delete [] resultado;
        }
        
        //delete [] vector;
        delete [] mat_aux_col;

        MPI_Finalize();
}
"""
text_file = open("MatrizxVeector.c", "w")
text_file.write(code)
text_file.close()

In [ ]:
!mpic++ -o MatrizxVeector MatrizxVeector.c

In [ ]:
!mpirun --allow-run-as-root -oversubscribe  -np 3 MatrizxVeector 